# Faster R-CNN 구현
### 텐서플로우 기반
### 주석은 한글 위주로 작성

In [1]:
import numpy as np
import os
import glob
import cv2
import xmltodict
import tensorflow as tf
import math
from tqdm import tqdm
from PIL import Image, ImageDraw

## 훈련 이미지 가져오기

In [2]:
train_x_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/train/VOCdevkit/VOC2007/JPEGImages'
train_y_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/train/VOCdevkit/VOC2007/Annotations'

test_x_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/test/VOCdevkit/VOC2007/JPEGImages'
test_y_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/test/VOCdevkit/VOC2007/Annotations'

In [3]:
list_train_x = sorted([x for x in glob.glob(train_x_path + '/**')])    
list_train_y = sorted([x for x in glob.glob(train_y_path + '/**')]) 

list_test_x = sorted([x for x in glob.glob(test_x_path + '/**')])    
list_test_y = sorted([x for x in glob.glob(test_y_path + '/**')]) 

print(len(list_train_x))
print(len(list_test_x))

5011
4952


훈련용 이미지는 5011개, 테스트용 이미지는 4952개가 있음을 알 수 있다. 그리고 둘다 x, y값이 존재한다. 

## 논문 흐름대로 코드 작성하려고 노력할 것

## 공용으로 사용하는 레이어 생성 
### ImageNet을 위해 훈련된 VGG16을 일부 가져온다

In [4]:
max_num = len(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers) # 레이어 최대 개수

SharedConvNet = tf.keras.models.Sequential()
for i in range(0, max_num-5):
    SharedConvNet.add(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers[i])

SharedConvNet.summary() # 13개의 레이어 공유(컨볼루션 레이어 10개, 풀링 레이어 3개). conv3_1까지만 가져온다고 한다는데 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

## RPN - 앵커 준비
#### 입력 이미지 기준으로 앵커를 생성한다.
#### 풀링을 3번 하므로 2^3 = 8이니 (8, 8)부터 (16, 8)...등 8픽셀식 중심 좌표를 옮겨가며 앵커들을 k개씩 생성한다
#### 생성한 앵커들 중 사용할 가치가 있는 앵커를 걸러낸다(이미지 범위를 벗어나지 않는 앵커들만 선정)
#### 선정한 앵커 중 실제 물체의 box와 얼마나 곂치는지(IoU) 계산해본다. 확실히 겹친다 하는 애들을 Positive 앵커로, 거의 안겹친다 하는 애들은 Negataive 앵커로 선정한다. 애매한 애들은 거른다.
#### 이렇게 생성된 앵커들로 미니배치를 생성한다. Positive 128개, Negative 128개로 만드는게 ideal한 구성이긴 한데 Positive한 앵커가 별로 없다. 그래서 Positive를 128개 못채웠으면 Negataive 앵커로 채워준다. 

In [5]:
n = 3 # 논문에서 n은 VGG16을 거치고 나온 특성맵 위를 슬라이딩 할 커널의 크기

# 생성할 앵커 크기는? 
anchor_size = [32, 64, 128] # 이미지 크기가 224*224라 32, 64, 128로 지정
anchor_aspect_ratio = [[1,1],[1,0.5], [0.5,1]] # W*L기준 


### 앵커를 만들어(224*224 기준) -> 여기서 positive 앵커, negative 앵커로 나눠 -> 얘들을 좀 비율 맞게 조합해서 256개의 앵커 그룹(미니배치)를 만들어 -> RPN훈련에 사용해

### 함수로 만드는 이유?
#### 이미지 별로 라벨에서 박스 좌표랑 객체 종류 뽑아내서 loss를 계산해야한다. 그래서 코드의 간결화?를 위해 함수를 만드는 것

In [6]:
# 앵커 생성 함수. 
def make_anchor(anchor_size, anchor_aspect_ratio, image_size = 224) :
    # 입력 이미지(그래봤자 224*224긴 하지만)에 맞춰 앵커를 생성해보자 

    anchors = [] # [x,y,w,h]로 이루어진 리스트 

    # 앵커 중심좌표 간격
    # 왜 간격이 8이냐? pooling을 3번 했기 때문에 각 특성맵의 픽셀 하나하나가 원래 이미지에서 8*8 픽셀을 대표하는 값들이다
    # 즉, 특성맵의 각 좌표 위를 슬라이딩 할건데 센터 좌표를 하나씩 건넌다는건 원래 이미지에서 8픽셀씩 움직인다는 것과 같다. 
    # 그렇기에 간격을 8*8로 설정한 것이다 
    interval_x = 8 
    interval_y = 8
    Center_max_x = 216 # 224 - 8, 중심좌표가 224가 될 수는 없다.
    Center_max_y = 216 # 224 - 8

    # 2단 while문 생성
    x = 8
    y = 8
    while(y <= 216):
        while(x <= 216):
            # k개의 앵커 생성. 여기서 k = len(anchor_size) * len(anchor_aspect_ratio)다
            for i in range(0, len(anchor_size)) : 
                for j in range(0, len(anchor_aspect_ratio)) :
                    anchor_width = anchor_aspect_ratio[j][0] * anchor_size[i]
                    anchor_height = anchor_aspect_ratio[j][1] * anchor_size[i]
                    # 얘를 사용할 수 있는 앵커인가? 필터링
                    if((x - (anchor_width/2) >= 0) and (y - (anchor_height/2) >= 0) and
                    (x + (anchor_width/2) <= 224) and (y + (anchor_height/2) <= 224)):
                        # 조건이 맞다고 판단되면 앵커 생성
                        anchor = [x, y, anchor_width, anchor_height]
                        anchors.append(anchor)
            x = x + interval_x 
        y = y + interval_y
        x = 8
    return anchors

In [7]:
anchors = make_anchor(anchor_size, anchor_aspect_ratio, 224)

len(anchors) # 4181개의 앵커 생성. 이 앵커 집합은 모든 이미지 입력에 적용할 앵커다

4181

#### 각 입력 이미지에서 객체의 Ground_Truth_Box, Class를 추출해야한다
#### Ground_Truth_Box는 RPN에서 IoU 구하는데 사용하기도 하고  Loss에서도 사용하기도 한다

#### Loss 사용을 위해 Class list도 만들자.
#### 논문의 loss 함수를 보면 pi가 있다. pi는 리스트인데 PASCAL VOC에 존재하는 객체들이 몇 %의 확률로 있느냐 나타내는거다
#### 이를 위해 어떤 객체들이 PASCAL VOC에 존재하는지 알아야한다. 

In [8]:
# label 추출을 위한 label 파일 리스트. label 정보가 xml파일 안에 있다.
xml_file_list = sorted([x for x in glob.glob(train_y_path + '/**')])

In [9]:
# 존재하는 객체 종류를 알아내자
def get_classes():
    classes = []

    for xml_file_path in xml_file_list: 

        f = open(xml_file_path)
        xml_file = xmltodict.parse(f.read())
        # 사진에 객체가 여러개 있을 경우
        try: 
            for obj in xml_file['annotation']['object']:
                classes.append(obj['name'].lower()) # 들어있는 객체 종류를 알아낸다
        # 사진에 객체가 하나만 있을 경우
        except TypeError as e: 
            classes.append(xml_file['annotation']['object']['name'].lower()) 
        f.close()

    classes = list(set(classes)) # set은 중복된걸 다 제거하고 유니크한? 아무튼 하나만 가져온다. 그걸 리스트로 만든다
    classes.sort() # 정렬

    return classes

In [10]:
label_classes = get_classes()

label_classes

['aeroplane',
 'bicycle',
 'bird',
 'boat',
 'bottle',
 'bus',
 'car',
 'cat',
 'chair',
 'cow',
 'diningtable',
 'dog',
 'horse',
 'motorbike',
 'person',
 'pottedplant',
 'sheep',
 'sofa',
 'train',
 'tvmonitor']

In [11]:
# IoU를 위한 Ground_Truth_Box와 Loss를 위한 Class 리스트를 얻는다. 
def get_label_fromImage(xml_file_path):

    f = open(xml_file_path)
    xml_file = xmltodict.parse(f.read()) 

    # 우선 원래 이미지 크기를 얻는다. 왜냐하면 앵커는 224*224 기준으로 만들었는데 원본 이미지는 224*224가 아니기 때문.
    # 224*224에 맞게 줄일려고 하는거다
    Image_Width = float(xml_file['annotation']['size']['height'])
    Image_Height  = float(xml_file['annotation']['size']['width'])


    Classes_list = [] 
    Ground_Truth_Box_list = [] 

    # multi-objects in image
    try:
        for obj in xml_file['annotation']['object']:
            obj_class = obj['name'].lower() 
            # 박스 좌표(왼쪽 위, 오른쪽 아래) 얻기
            x_min = float(obj['bndbox']['xmin']) 
            y_min = float(obj['bndbox']['ymin'])
            x_max = float(obj['bndbox']['xmax']) 
            y_max = float(obj['bndbox']['ymax'])

            # 224*224에 맞게 변형시켜줌
            x_min = float((224/Image_Width)*x_min)
            y_min = float((224/Image_Height)*y_min)
            x_max = float((224/Image_Width)*x_max)
            y_max = float((224/Image_Height)*y_max)


            # 얘들을 앵커랑 같은 양식으로 저장(x,y,w,h)
            Center_x = (x_min + x_max)/2
            Center_y = (y_min + y_max)/2
            Ground_Truth_Box_Width = x_max - x_min
            Ground_Truth_Box_Height = y_max - y_min

            Ground_Truth_Box = [Center_x, Center_y, Ground_Truth_Box_Width, Ground_Truth_Box_Height] 

            index = label_classes.index(obj_class) 

            Classes_list.append(index)
            Ground_Truth_Box_list.append(Ground_Truth_Box)

    # single-object in image
    except TypeError as e : 
        
        obj_class = xml_file['annotation']['object']['name'] 
        # 박스 좌표(왼쪽 위, 오른쪽 아래) 얻기
        x_min = float(xml_file['annotation']['object']['bndbox']['xmin']) 
        y_min = float(xml_file['annotation']['object']['bndbox']['ymin']) 
        x_max = float(xml_file['annotation']['object']['bndbox']['xmax']) 
        y_max = float(xml_file['annotation']['object']['bndbox']['ymax']) 

        # 224*224에 맞게 변형시켜줌
        x_min = float((224/Image_Width)*x_min)
        y_min = float((224/Image_Height)*y_min)
        x_max = float((224/Image_Width)*x_max)
        y_max = float((224/Image_Height)*y_max)


        # 얘들을 앵커랑 같은 양식으로 저장(x,y,w,h)
        Center_x = (x_min + x_max)/2
        Center_y = (y_min + y_max)/2
        Ground_Truth_Box_Width = x_max - x_min
        Ground_Truth_Box_Height = y_max - y_min

        Ground_Truth_Box = [Center_x, Center_y, Ground_Truth_Box_Width, Ground_Truth_Box_Height] 

        index = label_classes.index(obj_class) 

        Classes_list.append(index)
        Ground_Truth_Box_list.append(Ground_Truth_Box)


    return Classes_list, Ground_Truth_Box_list


In [17]:
# 테스트. xml_file_list[1]에 해당하는 이미지를 갖고 학습한다고 가정
Class_label, Ground_Truth_Box_list = get_label_fromImage(xml_file_list[1])
print(class_label) # car가 이미지에 있고
print(Ground_Truth_Box_list) # 224*224로 변환했을 때 약 (215, 85) 좌표에 (241, 125) 크기의 Ground_Truth_Box가 있음을 알 수 있다. 

[6]
[[215.59159159159162, 85.12, 241.48948948948953, 125.44]]


In [24]:
# 앵커들을 Positive, Negative 앵커로 나누자
def align_anchor(xml_file_path):
    Positive_Anchor = []
    Negative_Anchor = []

    # 이 함수에선 Ground_Truth_Box_List만 필요하다. 
    _, Ground_Truth_Box_list = get_label_fromImage(xml_file_path) 

    for i in range(0, len(anchors)): # 모든 앵커에 대한 IoU 계산
        # 각 앵커에 대해 존재하는 모든 Ground_Truth_Box의 IoU를 계산한다.
        # 어떤 객체든 IoU가 0.7이 넘어가면 바로 Positive 를 붙혀준다.
        IoU_list = [] 
        for j in range(0, len(Ground_Truth_Box_list)):
            # IoU 계산
            # 왼쪽 x좌표 기준으로는 ground_truth와 anchor 중 max를, 오른쪽 기준으로는 min값을 사용한다.
            # 그림을 그려서 왼쪽 좌표끼리 묶어서 max, 오른쪽 좌표끼리 묶어서 min을 선택해보면 이해가 쉬울거다.
            Ground_Truth_Box_min_x = Ground_Truth_Box_list[j][0] - (Ground_Truth_Box_list[j][2]/2)
            Ground_Truth_Box_min_y = Ground_Truth_Box_list[j][1] - (Ground_Truth_Box_list[j][3]/2)
            Ground_Truth_Box_max_x = Ground_Truth_Box_list[j][0] + (Ground_Truth_Box_list[j][2]/2)
            Ground_Truth_Box_max_y = Ground_Truth_Box_list[j][1] + (Ground_Truth_Box_list[j][3]/2)
            
            anchor_min_x = anchors[i][0] - (anchors[i][2]/2)
            anchor_min_y = anchors[i][1] - (anchors[i][3]/2)
            anchor_max_x = anchors[i][0] + (anchors[i][2]/2)
            anchor_max_y = anchors[i][1] + (anchors[i][3]/2)

            IoU_minX = max(anchor_min_x, Ground_Truth_Box_min_x)
            IoU_minY = max(anchor_max_x, Ground_Truth_Box_max_x)
            IoU_maxX = min(anchor_min_x, Ground_Truth_Box_min_x)
            IoU_maxY = min(anchor_min_x, Ground_Truth_Box_min_x)

            Intersection_Area = ((IoU_maxX - IoU_minX) * (IoU_maxY - IoU_minY)) # 교집합 넓이
            Union_Area = (anchors[i][2] * anchors[i][3]) + (Ground_Truth_Box_list[j][2] * Ground_Truth_Box_list[j][3]) - Intersection_Area # 합집합 넓이?

            IoU = Intersection_Area/Union_Area # IoU를 구했다

            IoU_list.append(IoU)

        # Ground_Truth_Box별로 IoU를 얻었다. 이제 Positive, Negative Anchor를 구분하자. 
        # Positive는 기준이 2개 있긴하지만 난 2번 기준(IoU가 0.7 이상)만 쓰겠다. any groud_truth_box에 대한 IoU가 0.7보다 크면 된다. 
        if max(IoU_list) > 0.7 : Positive_Anchor.append(anchors[i])
        # Negative는 IoU가 0.3보다 작은 앵커를 사용한다. 모든 Ground_Truth_Box에 대한 IoU가 0.3보다 작은 앵커가 negative 앵커다.
        elif max(IoU_list) < 0.3 : Negative_Anchor.append(anchors[i])
    
    return Positive_Anchor, Negative_Anchor



        



In [31]:
# 테스트
Positive_Anchor, Negative_Anchor = align_anchor(xml_file_list[1])
print(len(Positive_Anchor), ",", len(Negative_Anchor))

1875 , 1372


In [ ]:
# 앞서 얻은 정보를 이용해 Loss함수를 수행
def Loss():
    

loss 함수는 전체 Faster R-CNN에 대한 loss임. 
<br>
최종값으로 각 클래스에 대한 softmax(그래봤자 원-핫 인코딩)과 박스 위치인데 딥러닝 모델이 입력값(원래 이미지)와 출력값을 보고 갖고 있는 가중치, 필터를 '알아서' 학습해야함.
<br>
알아서 학습하는데 기준이? 되는 길잡이가 loss함수

RPN 생성. k마다 생성할 채널 갯수가 다르니 함수를 이용하자. 

In [6]:
k = 9

RPN_intermediate_layer = tf.keras.layers.Conv2D(512, (3,3), activation='relu',padding='same' ,input_shape=(28, 28, 512))
RPN_cls_Layer = tf.keras.layers.Conv2D(2*k, (1,1), activation='sigmoid', input_shape=(28, 28, 512)) # 클래스인가? 아닌가? 
RPN_reg_Layer = tf.keras.layers.Conv2D(4*k, (1,1), activation='relu', input_shape=(28, 28, 512))

이제 Fast R-CNN에 해당하는 Detection 구현 부분
<br>
Detection 부분은 RoI Pooling(7*7) -> 2개의 fully connected layers -> 클래스 분류 레이어와 박스 위치 분류 레이어에 하나씩 넣기

In [ ]:
# 여러가지 앵커를 ground_truth_box와 IoU 비교후 앵커들을 선정 -> RoI로 사용.
# RoI에 해당하는 영역을 특성맵에서 찾아야하는데 RoI는 224*224 기준이고 특성맵은 14*14임. -> 특성맵에 틀어맞게 RoI를 변환 -> 변환했으니 14*14 내부에 RoI가 변환되어 있음 -> 변환되어있는 RoI들을 FCs 2개에 넣어줌 -> 'class 레이어'와 'box 위치 분류 레이어' 두 곳에 넣어줌
# Roi Pooling만 잘 구현하면 되겠는데...어찌 구현할까(21/4/20)